**Purpose**: Analyzes the impact and effort required to fulfill CCPA consumer requests (Right to Know, Delete, or Opt-Out) across your data landscape


When a consumer submits a CCPA request, this function provides instant impact analysis to help compliance teams understand:
🎯 Request Scope Analysis

**Data Location:** Which tables contain the consumer's information
Record Volume: Estimated number of records to process
Data Categories: CCPA classification of affected data
Rights Applicability: Which consumer rights apply to each data type

⚖️ **Legal & Compliance Assessment**

**Legal Holds: **Identifies data that cannot be deleted due to legal requirements
**Retention Policies:** Compliance obligations that override deletion requests
**Business Justification:** Why certain data must be retained
Consumer Rights Limitations: When rights are restricted by law

📊 **Operational Impact Planning**

Effort Estimation: Hours required to fulfill the request
Complexity Assessment: Technical difficulty of data removal
System Dependencies: All affected systems and platforms
Automation Potential: What can be automated vs. manual review

🕐 **Timeline & Resource Management**

Compliance Deadline: Automatic 45-day CCPA deadline calculation
Priority Levels: High/Medium/Low complexity classification
Resource Allocation: Staff hours and skills needed

Function signature

In [0]:
%sql
-- Create the CCPA consumer request impact function 
CREATE OR REPLACE FUNCTION sumitsaraswat_catalog.compliance.ccpa_consumer_request_impact(
  consumer_identifier STRING DEFAULT 'example@email.com',
  identifier_type STRING DEFAULT 'email',
  request_type STRING DEFAULT 'delete'
)
RETURNS TABLE(
  affected_table STRING,
  ccpa_category STRING,
  estimated_record_count BIGINT,
  deletion_complexity STRING,
  legal_hold_status STRING,
  business_impact STRING,
  automated_deletion BOOLEAN,
  manual_review_required BOOLEAN,
  estimated_effort_hours DOUBLE,
  dependent_systems ARRAY<STRING>,
  compliance_deadline DATE,
  consumer_rights_applicable STRING,
  retention_override_reason STRING
)
RETURN (
  WITH consumer_data_analysis AS (
    -- Analyze tables that would contain the consumer's data
    SELECT 
      table_location,
      ccpa_category,
      personal_info_elements,
      sensitive_personal_info,
      sale_or_sharing_status,
      consumer_right_to_delete,
      consumer_right_to_opt_out,
      business_purpose,
      retention_period,
      
      -- Estimate record count based on table type and naming patterns
      CASE 
        WHEN table_location RLIKE '.*customer.*profile.*|.*user.*profile.*' THEN CAST(1 as BIGINT)
        WHEN table_location RLIKE '.*customer.*contact.*|.*customer.*communication.*|.*user.*contact.*' THEN CAST(25 as BIGINT)
        WHEN table_location RLIKE '.*customer.*|.*user.*' THEN CAST(5 as BIGINT)
        WHEN table_location RLIKE '.*transaction.*|.*order.*|.*payment.*|.*billing.*' THEN CAST(15 as BIGINT)
        WHEN table_location RLIKE '.*marketing.*|.*email.*|.*campaign.*' THEN CAST(75 as BIGINT)
        WHEN table_location RLIKE '.*analytics.*|.*tracking.*|.*log.*|.*event.*' THEN CAST(200 as BIGINT)
        WHEN table_location RLIKE '.*support.*|.*ticket.*|.*help.*' THEN CAST(8 as BIGINT)
        WHEN table_location RLIKE '.*financial.*|.*finance.*' THEN CAST(12 as BIGINT)
        ELSE CAST(3 as BIGINT)
      END as estimated_records,
      
      -- Assess deletion complexity based on multiple factors
      CASE 
        WHEN sensitive_personal_info = TRUE THEN 'High'
        WHEN table_location RLIKE '.*transaction.*|.*financial.*|.*billing.*|.*payment.*' THEN 'High'
        WHEN table_location RLIKE '.*analytics.*|.*log.*' AND 
             (CASE WHEN table_location RLIKE '.*marketing.*|.*email.*|.*campaign.*' THEN 75 ELSE 3 END) > 100 THEN 'Medium'
        WHEN ccpa_category = 'Commercial Information' THEN 'Medium'
        WHEN ccpa_category = 'Internet or Network Activity' THEN 'Medium'
        ELSE 'Low'
      END as complexity,
      
      -- Determine legal hold status based on table patterns and data types
      CASE 
        WHEN sensitive_personal_info = TRUE AND table_location RLIKE '.*tax.*|.*payroll.*' 
          THEN 'IRS 7-year retention requirement'
        WHEN table_location RLIKE '.*financial.*|.*transaction.*|.*billing.*|.*accounting.*' 
          THEN 'Subject to 7-year business records retention'
        WHEN table_location RLIKE '.*legal.*|.*compliance.*|.*audit.*' 
          THEN 'May be subject to litigation hold - legal review required'
        WHEN ccpa_category = 'Commercial Information' AND table_location RLIKE '.*contract.*|.*agreement.*'
          THEN 'Contract retention requirements may apply'
        WHEN table_location RLIKE '.*backup.*|.*archive.*'
          THEN 'Backup retention policies may prevent immediate deletion'
        ELSE 'No legal hold identified'
      END as legal_hold,
      
      -- Business impact assessment based on table purpose
      CASE 
        WHEN table_location RLIKE '.*financial.*|.*transaction.*|.*billing.*|.*revenue.*'
          THEN 'High - Financial records and reporting affected'
        WHEN table_location RLIKE '.*customer.*profile.*|.*user.*profile.*'
          THEN 'High - Core customer service capabilities impacted'
        WHEN table_location RLIKE '.*order.*|.*purchase.*|.*sales.*'
          THEN 'High - Sales and order management affected'
        WHEN table_location RLIKE '.*analytics.*|.*tracking.*|.*metrics.*'
          THEN 'Medium - Analytics and insights accuracy reduced'
        WHEN table_location RLIKE '.*support.*|.*ticket.*|.*help.*'
          THEN 'Medium - Support history and context lost'
        WHEN table_location RLIKE '.*marketing.*|.*campaign.*|.*email.*'
          THEN 'Low - Marketing effectiveness may decrease'
        WHEN table_location RLIKE '.*log.*|.*event.*'
          THEN 'Low - Historical logging affected'
        ELSE 'Low - Minimal business impact expected'
      END as biz_impact,
      
      -- Determine if automated deletion is possible
      CASE 
        WHEN sensitive_personal_info = TRUE THEN FALSE
        WHEN table_location RLIKE '.*financial.*|.*transaction.*|.*tax.*|.*payroll.*' THEN FALSE
        WHEN table_location RLIKE '.*legal.*|.*compliance.*|.*audit.*' THEN FALSE
        WHEN ccpa_category = 'Commercial Information' AND table_location RLIKE '.*contract.*' THEN FALSE
        WHEN (CASE 
                WHEN sensitive_personal_info = TRUE THEN 'High'
                WHEN table_location RLIKE '.*transaction.*|.*financial.*|.*billing.*' THEN 'High'
                ELSE 'Low'
              END) = 'High' THEN FALSE
        ELSE TRUE
      END as can_automate,
      
      -- Calculate effort estimation in hours
      CASE 
        WHEN (CASE 
                WHEN sensitive_personal_info = TRUE THEN 'High'
                WHEN table_location RLIKE '.*transaction.*|.*financial.*|.*billing.*' THEN 'High'
                WHEN ccpa_category = 'Commercial Information' THEN 'Medium'
                ELSE 'Low'
              END) = 'High' 
             AND legal_hold != 'No legal hold identified' THEN 12.0
        WHEN (CASE 
                WHEN sensitive_personal_info = TRUE THEN 'High'
                WHEN table_location RLIKE '.*transaction.*|.*financial.*|.*billing.*' THEN 'High'
                ELSE 'Low'
              END) = 'High' THEN 8.0
        WHEN (CASE 
                WHEN ccpa_category = 'Commercial Information' THEN 'Medium'
                WHEN ccpa_category = 'Internet or Network Activity' THEN 'Medium'
                ELSE 'Low'
              END) = 'Medium' 
             AND estimated_records > 50 THEN 6.0
        WHEN (CASE 
                WHEN ccpa_category = 'Commercial Information' THEN 'Medium'
                WHEN ccpa_category = 'Internet or Network Activity' THEN 'Medium'
                ELSE 'Low'
              END) = 'Medium' THEN 4.0
        WHEN estimated_records > 20 THEN 3.0
        ELSE 2.0
      END as effort_hours,
      
      -- Determine dependent systems based on table patterns
      CASE 
        WHEN table_location RLIKE '.*customer.*|.*user.*' 
          THEN ARRAY('Primary Database', 'CRM System', 'Customer Portal')
        WHEN table_location RLIKE '.*financial.*|.*billing.*|.*payment.*' 
          THEN ARRAY('Primary Database', 'Billing System', 'Payment Processor', 'Accounting System')
        WHEN table_location RLIKE '.*marketing.*|.*email.*|.*campaign.*' 
          THEN ARRAY('Primary Database', 'Email Platform', 'Marketing Automation', 'Analytics System')
        WHEN table_location RLIKE '.*analytics.*|.*tracking.*|.*metrics.*'
          THEN ARRAY('Primary Database', 'Analytics Platform', 'Data Warehouse', 'Reporting Tools')
        WHEN table_location RLIKE '.*support.*|.*ticket.*|.*help.*'
          THEN ARRAY('Primary Database', 'Support System', 'Ticketing Platform')
        WHEN table_location RLIKE '.*transaction.*|.*order.*|.*sales.*'
          THEN ARRAY('Primary Database', 'Order Management', 'Inventory System')
        ELSE ARRAY('Primary Database')
      END as systems,
      
      -- Determine applicable consumer rights based on request type
      CASE 
        WHEN request_type = 'delete' AND consumer_right_to_delete = FALSE 
          THEN 'Right to Delete LIMITED - Legal requirements prevent full deletion'
        WHEN request_type = 'delete' 
          THEN 'Right to Delete APPLICABLE - Full deletion required within 45 days'
        WHEN request_type = 'opt_out' AND consumer_right_to_opt_out = TRUE
          THEN 'Right to Opt-out APPLICABLE - Stop sale/sharing immediately'
        WHEN request_type = 'opt_out' AND consumer_right_to_opt_out = FALSE
          THEN 'Right to Opt-out NOT APPLICABLE - Data not sold or shared'
        WHEN request_type = 'know'
          THEN 'Right to Know APPLICABLE - Provide data categories and sources'
        ELSE 'Rights determination needed'
      END as rights_status,
      
      -- Retention override reasons for legal holds
      CASE 
        WHEN legal_hold LIKE '%IRS%' THEN 'Tax law requires 7-year retention'
        WHEN legal_hold LIKE '%business records%' THEN 'SOX and business continuity requirements'
        WHEN legal_hold LIKE '%litigation%' THEN 'Ongoing legal matter prevents deletion'
        WHEN legal_hold LIKE '%contract%' THEN 'Contractual obligations require retention'
        WHEN legal_hold LIKE '%backup%' THEN 'Technical: backup systems require separate deletion process'
        ELSE NULL
      END as retention_override
      
    FROM sumitsaraswat_catalog.compliance.ccpa_data_inventory()
    WHERE 
      -- Only include tables that would contain the specified identifier type
      CASE 
        WHEN identifier_type = 'email' THEN ARRAYS_OVERLAP(personal_info_elements, ARRAY('email'))
        WHEN identifier_type = 'phone' THEN ARRAYS_OVERLAP(personal_info_elements, ARRAY('phone'))
        WHEN identifier_type = 'name' THEN 
          (ARRAYS_OVERLAP(personal_info_elements, ARRAY('name')) OR 
           ARRAYS_OVERLAP(personal_info_elements, ARRAY('first name')) OR
           ARRAYS_OVERLAP(personal_info_elements, ARRAY('last name')))
        WHEN identifier_type = 'ssn' THEN ARRAYS_OVERLAP(personal_info_elements, ARRAY('ssn'))
        WHEN identifier_type = 'address' THEN ARRAYS_OVERLAP(personal_info_elements, ARRAY('address'))
        ELSE TRUE  -- Default: include all tables with personal info
      END
  )
  
  SELECT 
    table_location as affected_table,
    ccpa_category,
    estimated_records as estimated_record_count,
    complexity as deletion_complexity,
    legal_hold as legal_hold_status,
    biz_impact as business_impact,
    can_automate as automated_deletion,
    NOT can_automate as manual_review_required,
    effort_hours as estimated_effort_hours,
    systems as dependent_systems,
    CURRENT_DATE() + INTERVAL 45 DAYS as compliance_deadline,
    rights_status as consumer_rights_applicable,
    retention_override as retention_override_reason
  FROM consumer_data_analysis
  ORDER BY 
    CASE complexity 
      WHEN 'High' THEN 1 
      WHEN 'Medium' THEN 2 
      ELSE 3 
    END,
    estimated_records DESC
);

-- Create the summary view by category in your schema
CREATE OR REPLACE VIEW sumitsaraswat_catalog.compliance.ccpa_summary_by_category AS
SELECT 
  ccpa_category,
  COUNT(*) as table_count,
  SUM(CASE WHEN consumer_right_to_delete THEN 1 ELSE 0 END) as deletable_tables,
  SUM(CASE WHEN consumer_right_to_opt_out THEN 1 ELSE 0 END) as opt_out_required,
  SUM(CASE WHEN sensitive_personal_info THEN 1 ELSE 0 END) as sensitive_tables,
  COUNT(CASE WHEN implementation_gap != 'Implementation appears complete' THEN 1 END) as tables_with_gaps
FROM sumitsaraswat_catalog.compliance.ccpa_data_inventory()
GROUP BY ccpa_category
ORDER BY sensitive_tables DESC, table_count DESC;

-- Create the implementation gaps view in your schema
CREATE OR REPLACE VIEW sumitsaraswat_catalog.compliance.ccpa_implementation_gaps AS
SELECT 
  table_location,
  ccpa_category,
  implementation_gap,
  CASE 
    WHEN consumer_right_to_opt_out AND sale_or_sharing_status LIKE '%May be%' 
      THEN 'HIGH PRIORITY: Opt-out mechanism missing'
    WHEN sensitive_personal_info AND implementation_gap LIKE '%security%'
      THEN 'HIGH PRIORITY: Enhanced security needed'
    WHEN implementation_gap LIKE '%retention%'
      THEN 'MEDIUM PRIORITY: Retention policy needed'
    ELSE 'Standard Priority'
  END as priority_level,
  sensitive_personal_info
FROM sumitsaraswat_catalog.compliance.ccpa_data_inventory()
WHERE implementation_gap != 'Implementation appears complete'
ORDER BY 
  CASE priority_level
    WHEN 'HIGH PRIORITY: Opt-out mechanism missing' THEN 1
    WHEN 'HIGH PRIORITY: Enhanced security needed' THEN 2
    WHEN 'MEDIUM PRIORITY: Retention policy needed' THEN 3
    ELSE 4
  END,
  sensitive_personal_info DESC;

-- Test queries you can run to verify everything works:

-- 1. Test the main function
-- SELECT * FROM sumitsaraswat_catalog.compliance.ccpa_data_inventory() LIMIT 10;

-- 2. Get summary by category
-- SELECT * FROM sumitsaraswat_catalog.compliance.ccpa_summary_by_category;

-- 3. Check implementation gaps
-- SELECT * FROM sumitsaraswat_catalog.compliance.ccpa_implementation_gaps;

-- 4. See consumer request template
-- SELECT * FROM sumitsaraswat_catalog.compliance.ccpa_consumer_request_template;


Executing subquery: -- FUNCTION 2: CCPA Consumer Request Impact Analyzer (SQL Version)
-- This creates a simplified version that works without Python
CREATE OR REPLACE VIEW compliance.functions.ccpa_consumer_request_template AS
SELECT 
  'customers.main.customer_profiles' as affected_table,
  'Identifiers' as ccpa_category,
  CAST(1 as BIGINT) as record_count,
  'Low' as deletion_complexity,
  'No legal hold identified' as legal_hold_status,
  'Low - Standard customer data' as business_impact,
  TRUE as automated_deletion,
  FALSE as manual_review_required,
  CAST(2.0 as DOUBLE) as estimated_effort_hours,
  ARRAY('Primary Database', 'CRM System') as dependent_systems,
  CURRENT_DATE() + INTERVAL 45 DAYS as compliance_deadline
  
UNION ALL

SELECT 
  'sales.transactions.orders' as affected_table,
  'Commercial Information' as ccpa_category,
  CAST(23 as BIGINT) as record_count,
  'Medium' as deletion_complexity,
  'Subject to 7-year business records retention' as legal_hold_status,
  '

In [0]:
%sql
SELECT * FROM sumitsaraswat_catalog.compliance.ccpa_data_inventory() LIMIT 10;

table_location,ccpa_category,personal_info_elements,sale_or_sharing_status,consumer_right_to_know,consumer_right_to_delete,consumer_right_to_opt_out,business_purpose,third_party_recipients,retention_period,sensitive_personal_info,privacy_policy_disclosure,implementation_gap
sumitsaraswat_catalog.sample_data.financial_transactions,Commercial Information,List(credit_card_number),Never sold or shared (legal/service requirements only),true,true,false,"Payment processing, fraud prevention","Payment processors, Fraud detection services",90 days maximum (encrypted),true,Payment information used solely for transaction processing,Enhanced security controls verification needed
sumitsaraswat_catalog.sample_data.employee_records,Identifiers,List(social_security_number),Never sold or shared (legal/service requirements only),true,false,true,"Legal compliance, tax reporting, identity verification","Government agencies, Tax processors",7 years for business records,true,SSN collected only when legally required for tax/compliance,Enhanced security controls verification needed
sumitsaraswat_catalog.sample_data.customer_profiles,Identifiers,List(phone_number),"Not sold, shared with service providers only",true,true,false,"Customer service, account verification, security","Customer service platforms, SMS providers",24 months after last interaction,false,Phone numbers used for customer service and security verification,Implementation appears complete
sumitsaraswat_catalog.sample_data.customer_profiles,Identifiers,List(email_address),May be shared with marketing/analytics partners,true,true,true,"Customer communication, marketing, account management","Email service providers, Marketing platforms",24 months after last interaction,false,Email addresses collected for communication and marketing purposes,Implementation appears complete
sumitsaraswat_catalog.sample_data.customer_profiles,Identifiers,"List(first_name, last_name)","Not sold, shared with service providers only",true,true,false,"Account management, service delivery, legal compliance","Service providers, Payment processors",7 years for business records,false,Names used for account identification and service delivery,Implementation appears complete
sumitsaraswat_catalog.sample_data.customer_profiles,Identifiers,"List(street_address, city, zip_code)","Not sold, shared with service providers only",true,true,false,"Service delivery, billing, shipping","Shipping providers, Service technicians",24 months after last interaction,false,Addresses used for service delivery and billing,Implementation appears complete
sumitsaraswat_catalog.sample_data.employee_records,Identifiers,List(full_name),"Not sold, shared with service providers only",true,true,false,"Account management, service delivery, legal compliance","Service providers, Payment processors",7 years for business records,false,Names used for account identification and service delivery,Implementation appears complete
sumitsaraswat_catalog.sample_data.financial_transactions,Identifiers,List(customer_email),May be shared with marketing/analytics partners,true,true,true,"Customer communication, marketing, account management","Email service providers, Marketing platforms",24 months after last interaction,false,Email addresses collected for communication and marketing purposes,Implementation appears complete
sumitsaraswat_catalog.sample_data.marketing_campaigns,Identifiers,List(customer_email),May be shared with marketing/analytics partners,true,true,true,"Customer communication, marketing, account management","Email service providers, Marketing platforms",24 months after last interaction,false,Email addresses collected for communication and marketing purposes,Opt-out mechanism required for marketing data
sumitsaraswat_catalog.sample_data.marketing_campaigns,Identifiers,List(ip_address),"Not sold, shared with service providers only",true,true,false,"Service delivery, billing, shipping","Shipping providers, Service technicians",24 months after last int

In [0]:
%sql
-- 2. Get summary by category
 SELECT * FROM sumitsaraswat_catalog.compliance.ccpa_summary_by_category;

ccpa_category,table_count,deletable_tables,opt_out_required,sensitive_tables,tables_with_gaps
Commercial Information,2,2,0,2,2
Identifiers,9,8,4,1,2


### Example Usage

Request Types & Impact Analysis
🗑️ **Delete Requests** (request_type = 'delete')
Most Common & Complex Type
Analyzes:

Which tables contain deletable data
Legal restrictions preventing deletion
Business impact of removing customer data
Technical complexity of deletion process

**Key Outputs:**

**deletion_complexity:** Low/Medium/High
legal_hold_status: Legal requirements blocking deletion
retention_override_reason: Why data must be retained

🚫 **Opt-Out Requests** (request_type = 'opt_out')
Stop Sale/Sharing of Personal Information
Analyzes:

Which data is currently sold or shared
Systems requiring opt-out configuration
Marketing and analytics impact

**Key Outputs:**

consumer_rights_applicable: Whether opt-out applies
business_impact: Effect on marketing/analytics
dependent_systems: Platforms needing configuration



In [0]:
%sql
USE CATALOG sumitsaraswat_catalog;
USE SCHEMA compliance;

-- Analyze deletion request for a customer
SELECT 
    affected_table,
    deletion_complexity,
    estimated_effort_hours,
    legal_hold_status,
    compliance_deadline
FROM ccpa_consumer_request_impact('john.doe@example.com', 'email', 'delete')
ORDER BY deletion_complexity DESC, estimated_effort_hours DESC;

affected_table,deletion_complexity,estimated_effort_hours,legal_hold_status,compliance_deadline


In [0]:
%sql
-- Analyze opt-out request to stop data sale/sharing
SELECT 
    affected_table,
    ccpa_category,
    consumer_rights_applicable,
    business_impact
FROM ccpa_consumer_request_impact('user@company.com', 'email', 'opt_out')
WHERE consumer_rights_applicable LIKE '%APPLICABLE%'
ORDER BY business_impact DESC;

affected_table,ccpa_category,consumer_rights_applicable,business_impact
